In [1]:
import OpenBlender
import pandas as pd
import numpy as np
import json

In [4]:
token = '5fb0f21e951629785fb674bfSaw9f9kAXFtVMAXL11GpD2kdC2XNtG'
# Specify the action
action = 'API_getObservationsFromDataset'
interval = 60 * 60 * 24 # One day
parameters = { 
 'token' : token,
    'id_dataset':'5d4c39d09516290b01c8307b',
    'date_filter':{"start_date":"2017-01-01T06:00:00.000Z",
                   "end_date":"2020-11-10T06:00:00.000Z"},
    'aggregate_in_time_interval' : {
              'time_interval_size' : interval, 
              'output' : 'avg', 
              'empty_intervals' : 'impute'
      }
}
        
# Pull the data into a Pandas Dataframe
df = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']), convert_dates=False, convert_axes=False)
df.reset_index(drop=True, inplace=True)
df['date'] = [OpenBlender.unixToDate(ts, timezone = 'GMT') for ts in df.timestamp]
df = df.drop('timestamp', axis = 1)

Task ID: '5fb0f28c0895fafb4a9d8d4b'.
Total estimated consumption: 9974.48 processing units.
Task confirmed. Starting download..
33.33 %
66.67 %
100.0 % completed.


In [5]:
df.head()

price    high  count_agg     low        open  change  \
0  215.040000  217.19          1  214.03  216.800000   -0.19   
1  215.460000  219.18          1  215.11  218.100000   -0.97   
2  216.166667  218.77          0  214.46  216.546667    0.02   
3  216.873333  218.36          0  213.81  214.993333    1.01   
4  217.580000  217.95          1  213.16  213.440000    2.00   

                  date  
0  22-08-2018 00:00:00  
1  21-08-2018 00:00:00  
2  20-08-2018 00:00:00  
3  19-08-2018 00:00:00  
4  18-08-2018 00:00:00

In [6]:
df['timestamp'] = OpenBlender.dateToUnix(df['date'], 
                                           date_format = '%d-%m-%Y %H:%M:%S', 
                                           timezone = 'GMT')

# Now, let's search on OpenBlender
search_keyword = 'business'
df = df.sort_values('timestamp').reset_index(drop = True)
print('From : ' + OpenBlender.unixToDate(min(df.timestamp)))
print('Until: ' + OpenBlender.unixToDate(max(df.timestamp)))
# We need to pass our timestamp column and 
# search keywords as parameters.
OpenBlender.searchTimeBlends(token,
                             df.timestamp,
                             search_keyword)

From : 01-01-2017 00:00:00
Until: 10-11-2020 00:00:00


[{'name': 'Washington Post Business News Headlines',
  'url': 'https://www.openblender.io/#/dataset/explore/5d571a989516293a12ad4f4c',
  'id_dataset': '5d571a989516293a12ad4f4c',
  'features': ['headline', 'publisher', 'timestamp', 'title', 'url'],
  'num_observations': 791,
  'intersection': '100%',
  'description': 'Washington Post Business News Headlines daily headlines'},
 {'name': 'The Independent Business headlines',
  'url': 'https://www.openblender.io/#/dataset/explore/5d8660679516296ba7e6a432',
  'id_dataset': '5d8660679516296ba7e6a432',
  'features': ['headline', 'publisher', 'timestamp', 'title', 'url'],
  'num_observations': 6675,
  'intersection': '100%',
  'description': 'Daily news headlines of The Independent Business'},
 {'name': 'Belfast Telegraph Business headlines',
  'url': 'https://www.openblender.io/#/dataset/explore/5d884b739516294231c5961e',
  'id_dataset': '5d884b739516294231c5961e',
  'features': ['headline', 'publisher', 'timestamp', 'title', 'url'],
  'num_

In [7]:
positive_filter = {'name' : 'apple', 
                   'match_ngrams': ['apple', 'iphone', 'ipad', 
                                    'tim cook', 'mac store']}
# We need to pass the 'id_dataset', the name of the
# column (which is 'text') and an optional filter.
blend_source = {
                'id_dataset':'5e2ef74e9516294390e810a9',
                'feature' : 'text',
                'filter_text' : positive_filter
            }
df_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list')
df_anchor = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df_anchor

Task ID: '5fb0f2e60895fafb4a9d8d4c'.
Total estimated consumption: 56360.0 processing units.
Task confirmed. Starting download..
71.0%
100%


price        high  count_agg         low        open  change  \
0     116.15  116.330000          0  114.760000  115.800000    0.28   
1     116.15  116.330000          0  114.760000  115.800000    0.28   
2     116.15  116.330000          0  114.760000  115.800000    0.28   
3     116.15  116.330000          1  114.760000  115.800000    0.28   
4     116.02  116.510000          1  115.750000  115.850000   -0.11   
...      ...         ...        ...         ...         ...     ...   
1404  119.03  119.620000          1  116.880000  118.000000    3.55   
1405  118.69  119.200000          1  116.130000  118.410000   -0.29   
1406  117.90  120.126667          0  116.126667  119.106667   -0.86   
1407  117.11  121.053333          0  116.123333  119.803333   -1.43   
1408  116.32  121.980000          1  116.120000  120.500000   -2.00   

                     date     timestamp  \
0     01-01-2017 00:00:00  1.483229e+09   
1     02-01-2017 00:00:00  1.483315e+09   
2     03-01-2017 00:00:00  1.483402e+09   
3     04-01-2017 00:00:00  1.483488e+09   
4     05-01-2017 00:00:00  1.483574e+09   
...                   ...           ...   
1404  06-11-2020 00:00:00  1.604621e+09   
1405  07-11-2020 00:00:00  1.604707e+09   
1406  08-11-2020 00:00:00  1.604794e+09   
1407  09-11-2020 00:00:00  1.604880e+09   
1408  10-11-2020 00:00:00  1.604966e+09   

      WALL_STREE.text_COUNT_last1days:apple  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         1   
...                                     ...   
1404                                      0   
1405                                      0   
1406                                      0   
1407                                      0   
1408                                      0   

                        WALL_STREE.text_last1days:apple  
0                                                    []  
1                                                    []  
2                                                    []  
3                                                    []  
4     [apple confirms 1 billion investment in softba...  
...                                                 ...  
1404                                                 []  
1405                                                 []  
1406                                                 []  
1407                                                 []  
1408                                                 []  

[1409 rows x 10 columns]

In [8]:
action = 'API_createTextVectorizerPlus'
parameters = {
    'token' : token,
    'name' : 'Wall Street and USA Today Vectorizer',
    'sources':[
              {'id_dataset':"5e2ef74e9516294390e810a9", 
               'features' : ["text"]},
              {'id_dataset' : "5e32fd289516291e346c1726", 
               'features' : ["text"]}
    ],
    'ngram_range' : {'min' : 1, 'max' : 2},
    'language' : 'en',
    'remove_stop_words' : 'on',
    'min_count_limit' : 2
}
response = OpenBlender.call(action, parameters)
response

{'status': 'success',
 'num_ngrams': 4999,
 'response': 'Text Vectorizer created successfully.',
 'id_textVectorizer': '5fb0f300951629785fb674ce'}

In [14]:
blend_source = { 
                'id_textVectorizer':'5f74fb7895162967ca2160fe'
               }
df_blend = OpenBlender.timeBlend( token = "5fb0f378951629785ed1d0eap7jFIo0X6dT8ywpNkXxK9jxv9XoxkL",
                                  anchor_ts = df_anchor.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior_strict') .add_prefix('VEC.')
df_anchor = pd.concat([df_anchor, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df_anchor.head()

Task ID: '5fb0f3970895fafb4a9d8d4c'.
Total estimated consumption: 197268.38 processing units.
Task confirmed. Starting download..
25.0%
50.0%
75.0%
99.0%
100%


price    high  count_agg     low    open  change                 date  \
0  116.15  116.33          0  114.76  115.80    0.28  01-01-2017 00:00:00   
1  116.15  116.33          0  114.76  115.80    0.28  02-01-2017 00:00:00   
2  116.15  116.33          0  114.76  115.80    0.28  03-01-2017 00:00:00   
3  116.15  116.33          1  114.76  115.80    0.28  04-01-2017 00:00:00   
4  116.02  116.51          1  115.75  115.85   -0.11  05-01-2017 00:00:00   

      timestamp  WALL_STREE.text_COUNT_last1days:apple  \
0  1.483229e+09                                      0   
1  1.483315e+09                                      0   
2  1.483402e+09                                      0   
3  1.483488e+09                                      0   
4  1.483574e+09                                      1   

                     WALL_STREE.text_last1days:apple  ...  VEC.day week  \
0                                                 []  ...             0   
1                                                 []  ...             0   
2                                                 []  ...             0   
3                                                 []  ...             0   
4  [apple confirms 1 billion investment in softba...  ...             0   

   VEC.portion  VEC.lift  VEC.compete  VEC.pension  VEC.tensions  VEC.rural  \
0            0         0            0            0             0          0   
1            0         0            1            0             1          0   
2            0         0            0            0             0          0   
3            0         0            0            0             0          0   
4            0         0            0            0             0          0   

   VEC.accidents  VEC.oklahoma  VEC.richard  
0              0             0            0  
1              0             0            0  
2              0             0            0  
3              0             0            0  
4              0             0            0  

[5 rows x 4891 columns]

In [19]:
# Where ‘change’ decreased more than 0.5%
df_anchor['negative_poc'] = [1 if log_diff < -0.005 else 0 for log_diff in df_anchor['change']]
# Where ‘change’ increased more than 0.5%
df_anchor['positive_poc'] = [1 if log_diff > 0.005 else 0 for log_diff in df_anchor['change']]
# We need to shift our target one day so that we predict 
# for what will happen 'tomorrow' with news and data from 'today'
df_anchor['target'] = df_anchor['positive_poc'].shift(-1)
df_anchor[['change', 'positive_poc', 'target']]

change  positive_poc  target
0       0.28             1     1.0
1       0.28             1     1.0
2       0.28             1     1.0
3       0.28             1     0.0
4      -0.11             0     1.0
...      ...           ...     ...
1402    1.54             1     1.0
1403    4.08             1     1.0
1404    3.55             1     0.0
1405   -0.29             0     0.0
1406   -0.86             0     NaN

[1407 rows x 3 columns]

In [16]:
# Import libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
df_anchor = df_anchor[:-1] # Remove the last one with NaN
# Define target
target = 'target'
df_positive = df_anchor[1:].select_dtypes(['number'])
# Create train/test sets
X = df_positive.loc[:, df_positive.columns != target].values
y = df_positive.loc[:,[target]].values
div = int(round(len(X) * 0.71))
X_train = X[:div]
y_train = y[:div]
X_test = X[div:]
y_test = y[div:]
print('Train:')
print(X_train.shape)
print(y_train.shape)
print('Test:')
print(X_test.shape)
print(y_test.shape)

Train:
(998, 4887)
(998, 1)
Test:
(408, 4887)
(408, 1)


In [17]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

C:\Users\j3543\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [18]:
print("AUC score:")
print(roc_auc_score(y_test, y_pred))
print('---')
# Let's binarize and look at the confusion matrix
preds = [1 if val > 0.5 else 0 for val in y_pred]
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test, preds))
print('---')
# Lets look at the accuracy score
print('Accuracy:')
print(accuracy_score(y_test, preds))
print('---')

AUC score:
0.6594089038533484
---
Confusion Matrix:
[[ 85  80]
 [ 84 159]]
---
Accuracy:
0.5980392156862745
---
